In [ ]:

from google.colab import drive
drive.mount("/OCR")


Mounted at /OCR


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import pandas as pd
import cv2
import os
import imutils
from google.colab.patches import cv2_imshow
import pickle
import seaborn as sn
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sn
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import datetime
import platform


In [ ]:
!pip install easyocr


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 27.6 MB/s eta 0:00:00


In [ ]:
import easyocr
reader = easyocr.Reader(['en'], gpu=True)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.6/646.6 kB 7.4 MB/s eta 0:00:00


In [ ]:

from ultralytics import YOLO

# Load a model
model = YOLO('https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8x-pose-p6.pt')


100%|██████████| 190M/190M [00:00<00:00, 232MB/s]


In [ ]:
img=cv2.imread('/OCR/MyDrive/EasyOCR/3.jpg')
r=model.predict(img, save=False, imgsz=1280, conf=0.5)
j_r=r[0].tojson()

WARNING ⚠️ 'source' is missing. Using 'source=/usr/local/lib/python3.10/dist-packages/ultralytics/assets'.

WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/bus.jpg: 1280x960 4 persons, 161.2ms
image 2/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/zidane.jpg: 768x1280 2 persons, 199.6ms
Speed: 14.5ms preprocess, 180.4ms inference, 1055.4ms postprocess per image at shape (1, 3, 768, 1280)


In [ ]:

# img=cv2.imread('/OCR/MyDrive/EasyOCR/3_.jpg')
# r=model.predict(img, save=False, imgsz=1280, conf=0.90)
# a_f=r[0].plot()

# cv2_imshow(a_f)

In [ ]:

def find_polygon(image_path):
  keypoint_17=[]
  result = model.predict(image_path,imgsz=1280, conf=0.5)[0]
  print("asd : ",len(result.keypoints.data))
  vertices_all=[]
  for k in range(len(result.keypoints.data)):
    # print("iiiiiiiiiiiiiiiiii : ",k)
    keypoints = result.keypoints.xy[k].tolist()
    # bbox = result.boxes.xyxy[k].tolist()
    # print("boxes : ",bbox)
    keypoint_17.append(keypoints)
    # print(len(keypoints))
    left_shoulder=(0,0)
    right_shoulder=(0,0)
    right_hip=(0,0)
    left_hip=(0,0)
    for i, kp in enumerate(keypoints):
        x = int(kp[0])
        y = int(kp[1])
        if(i==5):
          left_shoulder=(x,y)
        if(i==6):
          right_shoulder=(x,y)
        if(i==11):
          left_hip=(x,y)
        if(i==12):
          right_hip=(x,y)
    # Define 4 points of the polygon (clockwise or counterclockwise order)
    v_tmp = np.array([left_shoulder,right_shoulder,right_hip,left_hip])
    vertices_all.append(v_tmp)
    # print("x : ",x,"     y: ",y,"    visibility : ",int(kp[2]))
  return vertices_all,keypoint_17

# vertices,keypoint_17=find_polygon("/OCR/MyDrive/EasyOCR/images/3_.jpg")
# # print(vertices)
# print(keypoint_17)

In [ ]:
# keypoint_17[2]

In [ ]:

def polygon_to_bbox(polygons):
    bbox_all=[]
    for polygon in polygons:
      # print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%",polygon)
      x_coordinates = [point[0] for point in polygon]
      y_coordinates = [point[1] for point in polygon]

      xmin = min(x_coordinates)
      ymin = min(y_coordinates)
      xmax = max(x_coordinates)
      ymax = max(y_coordinates)

      width = xmax - xmin
      height = ymax - ymin
      bbox_all.append((xmin, ymin, width, height))

    return bbox_all

# bbox=polygon_to_bbox(vertices)
# bbox

In [ ]:

def perform_ocr_on_image(img, coordinates_all):
    text_label=[]
    for coordinates in coordinates_all:
      # print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%",coordinates)
      x=coordinates[0]
      y=coordinates[1]
      w=coordinates[2]
      h=coordinates[3]
      # print(x,y,w,h)
      if 0 < x < img.shape[1] and 0 < y < img.shape[0] and 0 < x + w <= img.shape[1] and 0 < y + h <= img.shape[0]:
        cropped_img = img[y:y+h, x:x+w]

        # print(cropped_img)
        gray_img = cv2.cvtColor(cropped_img, cv2.COLOR_RGB2GRAY)
        results = reader.readtext(gray_img)
        # print("sdsdds",results)
        text = ""
        for res in results:
            if len(results) == 1 or (len(res[1]) > 6 and res[2] > 0.2):
                text = res[1]
        text_label.append(str(text))
    return text_label

# image_path = '/OCR/MyDrive/EasyOCR/3_.jpg'
# image = cv2.imread(image_path)
# ocr_results = perform_ocr_on_image(image, bbox)
# ocr_results

In [ ]:

def display_ocr_results(image, bounding_boxes, ocr_results):
    # Draw bounding boxes around keypoints
    print("display_ocr_results : ",ocr_results)
    for bbox in bounding_boxes:
        xmin, ymin, width, height = bbox
        cv2.rectangle(image, (xmin, ymin), (xmin + width, ymin + height), (0, 255, 0), 2)

    # Display OCR results on the image
    for i, (result, bbox) in enumerate(zip(ocr_results, bounding_boxes)):
        xmin, ymin, _, _ = bbox
        cv2.putText(image, f"{result}", (xmin, ymin - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0,255), 2, cv2.LINE_AA)

    # Display the image
    cv2_imshow( image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [ ]:
a=['', '21']
for i in range(len(a)):
  if(len(a[i])==0):
    a[i]="Need Face"
a

['Need Face', '21']

In [ ]:

df = pd.DataFrame(columns=['ketpoints_17'])

def Insert_into_pd_dataframe(indx,image_path):

  vertices,keypoint_17=find_polygon(image_path)

  bbox=[]
  # print("vertices : ",vertices)
  # print("keypoint_17 : ",keypoint_17)
  bbox.append(polygon_to_bbox(vertices))

  # Load the image for OCR
  # image = cv2.imread(image_path)
  # Perform OCR on the regions defined by keypoints
  ocr_results = perform_ocr_on_image(image_path, bbox[0])
  for i in range(len(ocr_results)):
    if(len(ocr_results[i])==0):
      ocr_results[i]="Face_"+str(i)
    else:
      ocr_results[i]="Player_"+str(ocr_results[i])
  d={}
  for i in range(len(ocr_results)):
    d.update({ocr_results[i]:keypoint_17[i]})
  df.loc[indx]=[d]



In [ ]:

video_path = "/OCR/MyDrive/football_detection/ball.mp4"
cap = cv2.VideoCapture(video_path)
total_frames = (cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("total_frames : ",total_frames)
success,image = cap.read()

c=0
b_name="Frame"
while(True):
  c=c+1
  success,image_ = cap.read()
  # cv2_imshow(image_)
  if success == False:
    break
  indx="Frame_"+str(c)
  Insert_into_pd_dataframe(indx,image_)

df

total_frames :  848.0

0: 768x1280 (no detections), 195.6ms
Speed: 6.9ms preprocess, 195.6ms inference, 5.9ms postprocess per image at shape (1, 3, 768, 1280)
asd :  1

0: 768x1280 (no detections), 151.5ms
Speed: 8.7ms preprocess, 151.5ms inference, 1.4ms postprocess per image at shape (1, 3, 768, 1280)
asd :  1

0: 768x1280 (no detections), 123.8ms
Speed: 6.6ms preprocess, 123.8ms inference, 1.6ms postprocess per image at shape (1, 3, 768, 1280)
asd :  1

0: 768x1280 (no detections), 125.0ms
Speed: 5.5ms preprocess, 125.0ms inference, 1.4ms postprocess per image at shape (1, 3, 768, 1280)
asd :  1

0: 768x1280 (no detections), 123.1ms
Speed: 3.7ms preprocess, 123.1ms inference, 1.4ms postprocess per image at shape (1, 3, 768, 1280)
asd :  1

0: 768x1280 (no detections), 123.4ms
Speed: 3.9ms preprocess, 123.4ms inference, 1.2ms postprocess per image at shape (1, 3, 768, 1280)
asd :  1

0: 768x1280 (no detections), 122.2ms
Speed: 5.4ms preprocess, 122.2ms inference, 1.3ms postprocess pe

,ketpoints_17
Frame_1,{}
Frame_2,{}
Frame_3,{}
Frame_4,{}
Frame_5,{}
...,...
Frame_843,{}
Frame_844,{}
Frame_845,{}
Frame_846,{}


In [ ]:

df.to_csv('player_keypoints.csv', encoding='utf-8')

In [ ]:
thisdict = {
}
thisdict.update({"color": "red"})

print(thisdict)

{'color': 'red'}
